In [37]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help user do research with their question on the files they upload.",
#     model="gpt-3.5-turbo-0125",
#     tools=[{"type": "file_search"}],
# )

# file_search = retriever

assistant_id = "asst_15XQiKFEspXqF3BzFMCrlYW6"


In [38]:
thread = client.beta.threads.create()

thread_id = thread.id

message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="I want you to help me with this file",
    attachments=[
        {
            "file_id": file.id,
            "tools": [
                {
                    "type": "file_search",
                }
            ],
        }
    ],
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (301079015.py, line 8)

In [ ]:
file = client.files.create(
    file = open("./files/chapter_one.txt", "rb"),
    purpose = "assistants"
)
file

In [29]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id,
)

run_id = run.id

In [35]:
import json

def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(message)
        # print(f"{message.role}:{message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source : {annotation.file_citation}")


def get_tool_output(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        # print(f"Calling function {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs 

def submit_tool_output(run_id, thread_id):
    outputs = get_tool_output(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs
    )

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content,
    )

In [ ]:
get_messages(thread_id)

In [ ]:
get_run(run_id, thread_id).status

In [ ]:
submit_tool_output(run_id, thread_id)

In [ ]:
send_message(thread_id, "Where dose he work?")

In [ ]:
stream.pirnt_messages()